<a href="https://colab.research.google.com/github/saschimi9/adm-1/blob/master/pe01/ADM1_ProgrammingExercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import gurobipy as grp
from gurobipy import GRB

Recall the **Scheduling Problem on Parallel Machines** from the Exercise Session 2. In this programming exercise, we will extend the time-indexed formulation to the case of non-identical machines. In this setting (known as **unrelated machines** in the litterature on scheduling theory), the processing time of a job depends on the machine on which it is executed; Each job has to be executed on exactly one machine, and the execution of job $j\in J=\{0,\ldots,n-1\}$ on machine $i\in M=\{0,\ldots,m-1\}$ requires $p_{i,j}\in\mathbb{N}$ units of processing time. Each machine can process at most one job at a time, and the jobs must be exectuded *non-preemtively*, i.e., jobs cannot be interrupted once started.

**Task:** Given some nonnegative weights $w_j$ ($\forall j\in J$) and some integer processing times $p_{i,j}$ ($\forall i,j \in M\times J$), find a *schedule* minimizing the weighted sum of completion time $\sum_{j\in J} w_j C_j$. Here, a schedule must specify the starting time $S_j\geq 0$ of each job $j\in J$, and the machine $M_j\in M$ on which it is executed.

Generate some unique data for the problem using the following cell, by replacing GROUP by the number of your homework group. Then, formulate the scheduling problem as an IP by adapting the **time-indexed formulation** of the Exercise Session 2. You can either use the gurobi framework that is pre-installed in this notebook, or download the data as a .json file and solve the problem with your favorite solver or interface (cvxpy with any solver, pyscipopt, ...).

Return the solution via ISIS by outputing the solution in the following form: 

``OPT_IP {0: (M0,S0),, 1: (M1,S1), 2: (M2,S2), ... }``,

where
  * ``OPT_IP`` is the value of the optimal objective function of the problem, i.e., $\sum_{j\in J} w_j C_j$
  * ``Sj`` is the starting time of job $j$, for $j=0,...,n-1$
  * ``Mj`` is the machine executing job $j$, for $j=0,...,n-1$.

**Hint** You will need to introduce variables ``x[i,j,t]`` for several triples $(i,j,t)$ to indicate that job $j$ starts (or completes, as you like) on machine $i$ at time $t$. A bound for the time horizon of the problem is $T=\lceil (\frac{n}{m}+1) p_{\max} \rceil$, where $p_{\max}$ is an upper bound for all processing times (*why?*). 


In [2]:


my_group = 9

#function to get a seed for your group # DO NOT CHANGE THIS FUNCTION !
def get_seed_for_my_group(group_number):
  return list(range(1,41))[group_number]


#seed the pseudo-random number generator with your group number
seed = get_seed_for_my_group(my_group)
np.random.seed(seed)

#generate the data
n=14                                  # number of jobs
m=4                                   # number of machines
pmax=7                                # processing times are between 1 and pmax
w = np.random.randint(1,101,n)        # w[j] is the weight of job j
p = np.random.randint(1,pmax+1,(m,n)) # p[i,j] is the processing time 
                                      #        of job j on machine i         

#uncomment the following block to save the data as a .json file that you can
#use with your favorite software
"""
from google.colab import files
import json

data = {'n':n,
        'm':m,
        'pmax': pmax,
        'p': {i: {j: int(p[i,j]) for j in range(n)} for i in range(m)},
        'w': {j: int(w[j]) for j in range(n)}
        }
with open('ADM1_prog_ex2_data.json','w') as fp:
  json.dump(data, fp, indent=4)

files.download('ADM1_prog_ex2_data.json')
"""

"\nfrom google.colab import files\nimport json\n\ndata = {'n':n,\n        'm':m,\n        'pmax': pmax,\n        'p': {i: {j: int(p[i,j]) for j in range(n)} for i in range(m)},\n        'w': {j: int(w[j]) for j in range(n)}\n        }\nwith open('ADM1_prog_ex2_data.json','w') as fp:\n  json.dump(data, fp, indent=4)\n\nfiles.download('ADM1_prog_ex2_data.json')\n"

Here is a minimal working example to introduce the syntax. In this program, we find $\max\{x_1+x_2+y\ \colon \ y=0.5,\ x_1,x_2\leq 1\}$.

Replace ``MAXIMIZE`` by ``MINIMIZE``. What do you expect will happen? What happens?

In [3]:
M = grp.Model("Test")#name of model

# Variables
x= M.addVars(2, vtype=GRB.INTEGER)#the first input is the number of coefficients of variable
y=M.addVars(1, vtype=GRB.CONTINUOUS)#real number


# Objective function
M.setObjective(x[0]+x[1]+y[0], \
GRB.MAXIMIZE)

# Constraints
M.addConstr(y[0]==0.5)
M.addConstrs((x[i] <= 1 for i in range(2)), name='c') #note that name is optional.


M.optimize()


Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xd75aac66
Variable types: 1 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 2.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.500000000000e+00, best bound 2.500000000000e+00, gap 0.0000%


In [12]:
M = grp.Model("Time-indexed parallel machine job scheduling")#name of model

T = int(np.ceil(pmax*(n/m + 1)))
# Variables
x = M.addVars(n, T, m, vtype=GRB.BINARY)

# Constraints
for m_tild in range(m):
    for j in range(n):
        sumlist = [x[j, t, m_tild] for t in range(p[m_tild, j], T)]
        M.addConstr(grp.quicksum(sumlist) == 1, name=f'Job {j} single start')

for m_tild in range(m):
    for t in range(T):
        sumlist = []
        for j in range(n):
            for s in range(t, min(t + p[m_tild, j]-1, T)):
                sumlist.append(x[j, s, m_tild])
        M.addConstr(grp.quicksum(sumlist) <= 1, name=f'One job at each time step of machine {m}')

# Objective function
cost = 0
for m_tild in range(m):
    for t in range(T):
        for j in range(n):
            cost += w[j]*t*x[j, t, m_tild]     
M.setObjective(cost, GRB.MINIMIZE)

In [13]:
M.optimize()
if M.Status == GRB.OPTIMAL:
    print('Optimal objective: %g' % M.ObjVal)
elif M.Status == GRB.INF_OR_UNBD:
    print('M is infeasible or unbounded')
    sys.exit(0)
elif M.Status == GRB.INFEASIBLE:
    print('M is infeasible')
    sys.exit(0)
elif M.Status == GRB.UNBOUNDED:
    print('M is unbounded')
    sys.exit(0)
else:
    print('Optimization ended with status %d' % M.Status)
    sys.exit(0)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 184 rows, 1792 columns and 6961 nonzeros
Model fingerprint: 0xa80c6ca1
Variable types: 0 continuous, 1792 integer (1792 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 16 rows and 484 columns
Presolve time: 0.05s
Presolved: 168 rows, 1308 columns, 6030 nonzeros
Variable types: 0 continuous, 1308 integer (1308 binary)

Root relaxation: infeasible, 249 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -   

SystemExit: 0

/home/saschimi/projects/adm-1/adm/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


You can implement you IP/LP in the following cells.